In [15]:
pip install pandas openpyxl


Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd

# Load Excel file
file_path = "C:/Users/tanupriya/Desktop/datasets/bank.xls"

# Load sheets
bank_data = pd.read_excel(file_path, sheet_name="bank statement")
ledger_data = pd.read_excel(file_path, sheet_name="internal ledger")

# Display the loaded data
print("Bank Data:")
print(bank_data.head())
print("\nLedger Data:")
print(ledger_data.head())


Bank Data:
        Date Description  Amount  Account Number
0 2024-12-01     Deposit    5000           12345
1 2024-12-02     Payment   -1500           12345
2 2024-12-03    Transfer    3000           12345
3 2024-12-05     Payment   -2000           12345

Ledger Data:
        Date Transaction ID  Amount    Vendor  Account
0 2024-12-01           T001    5000      Bank    12345
1 2024-12-02           T002   -1500  Vendor A    12345
2 2024-12-03           T003    3000  Vendor B    12345
3 2024-12-04           T004   -2500  Vendor C    12345


In [17]:
# Convert Date columns to datetime format
bank_data['Date'] = pd.to_datetime(bank_data['Date'], errors='coerce')
ledger_data['Date'] = pd.to_datetime(ledger_data['Date'], errors='coerce')

# Check for missing values
print("\nMissing Values in Bank Data:")
print(bank_data.isnull().sum())
print("\nMissing Values in Ledger Data:")
print(ledger_data.isnull().sum())

# Remove duplicate rows if needed
bank_data = bank_data.drop_duplicates()
ledger_data = ledger_data.drop_duplicates()

# Display cleaned data
print("\nCleaned Bank Data:")
print(bank_data.head())
print("\nCleaned Ledger Data:")
print(ledger_data.head())



Missing Values in Bank Data:
Date              0
Description       0
Amount            0
Account Number    0
dtype: int64

Missing Values in Ledger Data:
Date              0
Transaction ID    0
Amount            0
Vendor            0
Account           0
dtype: int64

Cleaned Bank Data:
        Date Description  Amount  Account Number
0 2024-12-01     Deposit    5000           12345
1 2024-12-02     Payment   -1500           12345
2 2024-12-03    Transfer    3000           12345
3 2024-12-05     Payment   -2000           12345

Cleaned Ledger Data:
        Date Transaction ID  Amount    Vendor  Account
0 2024-12-01           T001    5000      Bank    12345
1 2024-12-02           T002   -1500  Vendor A    12345
2 2024-12-03           T003    3000  Vendor B    12345
3 2024-12-04           T004   -2500  Vendor C    12345


In [18]:
# Merge the datasets
merged_data = pd.merge(
    bank_data,
    ledger_data,
    on=["Date", "Amount"],
    how="outer",
    indicator=True
)

# Display merged data
print(merged_data.head())


        Date Description  Amount  Account Number Transaction ID    Vendor  \
0 2024-12-01     Deposit    5000         12345.0           T001      Bank   
1 2024-12-02     Payment   -1500         12345.0           T002  Vendor A   
2 2024-12-03    Transfer    3000         12345.0           T003  Vendor B   
3 2024-12-05     Payment   -2000         12345.0            NaN       NaN   
4 2024-12-04         NaN   -2500             NaN           T004  Vendor C   

   Account      _merge  
0  12345.0        both  
1  12345.0        both  
2  12345.0        both  
3      NaN   left_only  
4  12345.0  right_only  


In [19]:
# Separate matched and unmatched transactions
matched = merged_data[merged_data["_merge"] == "both"]
unmatched_bank = merged_data[merged_data["_merge"] == "left_only"]
unmatched_ledger = merged_data[merged_data["_merge"] == "right_only"]

# Display results
print("\nMatched Transactions:")
print(matched)
print("\nUnmatched Transactions in Bank Data:")
print(unmatched_bank)
print("\nUnmatched Transactions in Ledger Data:")
print(unmatched_ledger)



Matched Transactions:
        Date Description  Amount  Account Number Transaction ID    Vendor  \
0 2024-12-01     Deposit    5000         12345.0           T001      Bank   
1 2024-12-02     Payment   -1500         12345.0           T002  Vendor A   
2 2024-12-03    Transfer    3000         12345.0           T003  Vendor B   

   Account _merge  
0  12345.0   both  
1  12345.0   both  
2  12345.0   both  

Unmatched Transactions in Bank Data:
        Date Description  Amount  Account Number Transaction ID Vendor  \
3 2024-12-05     Payment   -2000         12345.0            NaN    NaN   

   Account     _merge  
3      NaN  left_only  

Unmatched Transactions in Ledger Data:
        Date Description  Amount  Account Number Transaction ID    Vendor  \
4 2024-12-04         NaN   -2500             NaN           T004  Vendor C   

   Account      _merge  
4  12345.0  right_only  


In [22]:
# Save the results to Excel
with pd.ExcelWriter("reconciliation_report.xlsx") as writer:
    matched.to_excel(writer, sheet_name="Matched", index=False)
    unmatched_bank.to_excel(writer, sheet_name="Unmatched Bank", index=False)
    unmatched_ledger.to_excel(writer, sheet_name="Unmatched Ledger", index=False)

print("Reconciliation report saved as 'reconciliation_report.xlsx'")


Reconciliation report saved as 'reconciliation_report.xlsx'


In [23]:
# Summary counts
summary = {
    "Total Bank Transactions": len(bank_data),
    "Total Ledger Transactions": len(ledger_data),
    "Matched Transactions": len(matched),
    "Unmatched in Bank": len(unmatched_bank),
    "Unmatched in Ledger": len(unmatched_ledger),
}

print("\nReconciliation Summary:")
for key, value in summary.items():
    print(f"{key}: {value}")



Reconciliation Summary:
Total Bank Transactions: 4
Total Ledger Transactions: 4
Matched Transactions: 3
Unmatched in Bank: 1
Unmatched in Ledger: 1


In [24]:
import pandas as pd

# Sample Compliance Checks

# Check for missing Transaction IDs in the ledger_data
ledger_data['Transaction_ID_Valid'] = ledger_data['Transaction ID'].notnull()

# Check for large transactions above $10,000 without justification in ledger_data
ledger_data['Large_Transaction_No_Justification'] = ledger_data.apply(
    lambda x: "Non-Compliant" if x['Amount'] > 10000 and pd.isna(x['Justification']) else "Compliant", axis=1
)

# Check for negative amounts in bank_data (assuming only deposits should be positive)
bank_data['Negative_Amount'] = bank_data['Amount'] < 0

# Display results
print("Compliance Check Results:")
print(ledger_data[['Transaction ID', 'Amount', 'Transaction_ID_Valid', 'Large_Transaction_No_Justification']].head())
print(bank_data[['Date', 'Amount', 'Negative_Amount']].head())


Compliance Check Results:
  Transaction ID  Amount  Transaction_ID_Valid  \
0           T001    5000                  True   
1           T002   -1500                  True   
2           T003    3000                  True   
3           T004   -2500                  True   

  Large_Transaction_No_Justification  
0                          Compliant  
1                          Compliant  
2                          Compliant  
3                          Compliant  
        Date  Amount  Negative_Amount
0 2024-12-01    5000            False
1 2024-12-02   -1500             True
2 2024-12-03    3000            False
3 2024-12-05   -2000             True


In [25]:
# Save compliance results
with pd.ExcelWriter("compliance_check_report.xlsx") as writer:
    ledger_data.to_excel(writer, sheet_name="Ledger Compliance", index=False)
    bank_data.to_excel(writer, sheet_name="Bank Compliance", index=False)

print("Compliance report saved as 'compliance_check_report.xlsx'")


Compliance report saved as 'compliance_check_report.xlsx'
